# NYC Bike Rhythms — Data Pipeline

This notebook downloads and processes Citi Bike trip data to generate JSON files for the visualization.

**Output files:**
- `neighborhoods.json` — NYC neighborhood boundaries with metadata
- `weekly-patterns.json` — Hourly ride patterns by neighborhood
- `story-moments.json` — Pre-computed statistics for story beats
- `metadata.json` — Data summary

**Runtime:** ~15-20 minutes for full year

## 1. Setup

In [ ]:
# Install required packages
!pip install -q pandas geopandas shapely requests tqdm

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import zipfile
import os
import json
from io import BytesIO
from shapely.geometry import Point
from tqdm import tqdm
from google.colab import files

# Create output directory
os.makedirs('output', exist_ok=True)
os.makedirs('raw', exist_ok=True)

print('Setup complete!')

## 2. Download NYC Neighborhood Boundaries

In [ ]:
# Download NTA (Neighborhood Tabulation Area) boundaries
NTA_URL = 'https://data.cityofnewyork.us/api/geospatial/9nt8-h7nd?method=export&format=GeoJSON'

print('Downloading NYC neighborhood boundaries...')
nta = gpd.read_file(NTA_URL)
print(f'Loaded {len(nta)} neighborhoods')

# Preview
nta[['ntaname', 'ntacode', 'boroname']].head(10)

## 3. Download Citi Bike Trip Data (Full Year 2025)

In [ ]:
# Configuration
YEAR = 2025
MONTHS = range(1, 13)  # All 12 months

BASE_URL = 'https://s3.amazonaws.com/tripdata'

def download_month(year, month):
    """Download and extract a single month of Citi Bike data."""
    filename = f'{year}{month:02d}-citibike-tripdata.zip'
    url = f'{BASE_URL}/{filename}'
    
    print(f'Downloading {filename}...')
    response = requests.get(url, stream=True)
    
    if response.status_code != 200:
        print(f'  Failed to download {filename}')
        return None
    
    # Extract CSV from ZIP
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        
        # Read all CSVs in the ZIP (some months have multiple files)
        dfs = []
        for csv_file in csv_files:
            with z.open(csv_file) as f:
                df = pd.read_csv(f, low_memory=False)
                dfs.append(df)
        
        combined = pd.concat(dfs, ignore_index=True)
        print(f'  Loaded {len(combined):,} trips')
        return combined

# Download all months
all_trips = []
for month in MONTHS:
    df = download_month(YEAR, month)
    if df is not None:
        all_trips.append(df)

# Combine all months
trips = pd.concat(all_trips, ignore_index=True)
print(f'\nTotal trips: {len(trips):,}')

## 4. Data Cleaning & Feature Engineering

In [ ]:
# Standardize column names (they vary between months)
column_mapping = {
    'ride_id': 'ride_id',
    'rideable_type': 'rideable_type',
    'started_at': 'started_at',
    'ended_at': 'ended_at',
    'start_station_name': 'start_station_name',
    'start_station_id': 'start_station_id',
    'end_station_name': 'end_station_name',
    'end_station_id': 'end_station_id',
    'start_lat': 'start_lat',
    'start_lng': 'start_lng',
    'end_lat': 'end_lat',
    'end_lng': 'end_lng',
    'member_casual': 'member_casual'
}

# Rename columns to standard names
trips = trips.rename(columns={k: v for k, v in column_mapping.items() if k in trips.columns})

# Parse timestamps
trips['started_at'] = pd.to_datetime(trips['started_at'])
trips['ended_at'] = pd.to_datetime(trips['ended_at'])

# Extract time features
trips['hour'] = trips['started_at'].dt.hour
trips['day_of_week'] = trips['started_at'].dt.dayofweek  # 0=Monday, 6=Sunday
trips['month'] = trips['started_at'].dt.month
trips['is_weekend'] = trips['day_of_week'].isin([5, 6])
trips['is_member'] = trips['member_casual'] == 'member'

# Remove trips with missing coordinates
before = len(trips)
trips = trips.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng'])
print(f'Removed {before - len(trips):,} trips with missing coordinates')
print(f'Remaining trips: {len(trips):,}')

## 5. Spatial Join: Map Stations to Neighborhoods

In [ ]:
# Get unique stations
start_stations = trips[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
start_stations.columns = ['station_id', 'station_name', 'lat', 'lng']

end_stations = trips[['end_station_id', 'end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
end_stations.columns = ['station_id', 'station_name', 'lat', 'lng']

stations = pd.concat([start_stations, end_stations]).drop_duplicates(subset='station_id')
print(f'Unique stations: {len(stations):,}')

# Create GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations.lng, stations.lat),
    crs='EPSG:4326'
)

# Spatial join to NTA
stations_with_nta = gpd.sjoin(
    stations_gdf,
    nta[['ntaname', 'ntacode', 'boroname', 'geometry']],
    how='left',
    predicate='within'
)

# Create station -> NTA lookup
station_nta_lookup = stations_with_nta.set_index('station_id')[['ntacode', 'ntaname', 'boroname']].to_dict('index')

# Count stations per NTA
nta_station_counts = stations_with_nta.groupby('ntacode').size()
print(f'\nNeighborhoods with Citi Bike stations: {len(nta_station_counts)}')

In [ ]:
# Add NTA codes to trips
def get_nta(station_id):
    if pd.isna(station_id):
        return None
    info = station_nta_lookup.get(station_id)
    return info['ntacode'] if info else None

print('Mapping trips to neighborhoods...')
trips['start_nta'] = trips['start_station_id'].map(lambda x: get_nta(x))
trips['end_nta'] = trips['end_station_id'].map(lambda x: get_nta(x))

# Remove trips that couldn't be mapped
before = len(trips)
trips = trips.dropna(subset=['start_nta'])
print(f'Removed {before - len(trips):,} trips not in any neighborhood')
print(f'Final trip count: {len(trips):,}')

## 6. Aggregate Data

In [ ]:
# Aggregate departures by NTA, day of week, hour
departures = trips.groupby(['start_nta', 'day_of_week', 'hour']).agg(
    departures=('ride_id', 'count'),
    member_departures=('is_member', 'sum'),
).reset_index()

# Aggregate arrivals
arrivals = trips.groupby(['end_nta', 'day_of_week', 'hour']).agg(
    arrivals=('ride_id', 'count'),
).reset_index()
arrivals.columns = ['nta', 'day_of_week', 'hour', 'arrivals']

# Merge departures and arrivals
departures.columns = ['nta', 'day_of_week', 'hour', 'departures', 'member_departures']
hourly_stats = departures.merge(arrivals, on=['nta', 'day_of_week', 'hour'], how='outer').fillna(0)

# Calculate net flow
hourly_stats['net_flow'] = hourly_stats['arrivals'] - hourly_stats['departures']
hourly_stats['member_pct'] = (hourly_stats['member_departures'] / hourly_stats['departures'] * 100).fillna(0)

print(f'Aggregated {len(hourly_stats):,} rows (NTA × day × hour)')

## 7. Exploratory Analysis

In [ ]:
# Analysis A: Which neighborhoods wake up first?
weekday_morning = hourly_stats[
    (hourly_stats['day_of_week'] < 5) & 
    (hourly_stats['hour'] >= 5) & 
    (hourly_stats['hour'] <= 9)
].groupby(['nta', 'hour'])['departures'].mean().reset_index()

# Find peak hour for each neighborhood
peak_hours = hourly_stats[
    hourly_stats['day_of_week'] < 5  # Weekdays
].groupby('nta').apply(
    lambda x: x.loc[x['departures'].idxmax(), 'hour'] if len(x) > 0 else None
).reset_index(name='peak_hour')

print('Neighborhoods with earliest peak (6-7am):')
early_risers = peak_hours[peak_hours['peak_hour'] <= 7]
for _, row in early_risers.head(5).iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: {int(row['peak_hour'])}:00")

print('\nNeighborhoods with latest peak (after 9pm):')
night_owls = peak_hours[peak_hours['peak_hour'] >= 21]
for _, row in night_owls.head(5).iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: {int(row['peak_hour'])}:00")

In [ ]:
# Analysis B: Morning rush - which neighborhoods are exporters vs importers?
morning_rush = hourly_stats[
    (hourly_stats['day_of_week'] < 5) & 
    (hourly_stats['hour'] >= 8) & 
    (hourly_stats['hour'] <= 9)
].groupby('nta').agg(
    total_departures=('departures', 'sum'),
    total_arrivals=('arrivals', 'sum'),
    net_flow=('net_flow', 'sum')
).reset_index()

# Top exporters (residential areas)
print('Top 5 EXPORTERS (residential areas - people leaving for work):')
exporters = morning_rush.nsmallest(5, 'net_flow')
for _, row in exporters.iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: {int(row['net_flow']):,} net flow")

# Top importers (office districts)
print('\nTop 5 IMPORTERS (office districts - people arriving for work):')
importers = morning_rush.nlargest(5, 'net_flow')
for _, row in importers.iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: +{int(row['net_flow']):,} net flow")

In [ ]:
# Analysis C: Friday night vs Monday night
friday_night = hourly_stats[
    (hourly_stats['day_of_week'] == 4) &  # Friday
    (hourly_stats['hour'] >= 22)
].groupby('nta')['arrivals'].sum()

monday_night = hourly_stats[
    (hourly_stats['day_of_week'] == 0) &  # Monday
    (hourly_stats['hour'] >= 22)
].groupby('nta')['arrivals'].sum()

night_comparison = pd.DataFrame({
    'friday': friday_night,
    'monday': monday_night
}).fillna(0)
night_comparison['ratio'] = night_comparison['friday'] / night_comparison['monday'].replace(0, 1)

print('Neighborhoods with BIGGEST Friday night spike (vs Monday):')
for nta, row in night_comparison.nlargest(5, 'ratio').iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == nta]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == nta]) > 0 else nta
    print(f"  {nta_name}: {row['ratio']:.1f}x more arrivals on Friday")

In [ ]:
# Analysis D: Weekend vs weekday transformation
weekday_total = hourly_stats[
    hourly_stats['day_of_week'] < 5
].groupby('nta')['departures'].sum()

weekend_total = hourly_stats[
    hourly_stats['day_of_week'] >= 5
].groupby('nta')['departures'].sum()

weekend_comparison = pd.DataFrame({
    'weekday': weekday_total / 5,  # Per day average
    'weekend': weekend_total / 2   # Per day average
}).fillna(0)
weekend_comparison['ratio'] = weekend_comparison['weekend'] / weekend_comparison['weekday'].replace(0, 1)

print('Neighborhoods that GROW most on weekends (likely parks/recreation):')
for nta, row in weekend_comparison.nlargest(5, 'ratio').iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == nta]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == nta]) > 0 else nta
    print(f"  {nta_name}: {row['ratio']:.1f}x weekend vs weekday")

print('\nNeighborhoods that SHRINK most on weekends (likely office districts):')
for nta, row in weekend_comparison.nsmallest(5, 'ratio').iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == nta]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == nta]) > 0 else nta
    print(f"  {nta_name}: {row['ratio']:.2f}x weekend vs weekday")

In [ ]:
# Analysis E: Member % by neighborhood (tourist vs local)
member_pct_by_nta = hourly_stats.groupby('nta').agg(
    total_departures=('departures', 'sum'),
    member_departures=('member_departures', 'sum')
).reset_index()
member_pct_by_nta['member_pct'] = member_pct_by_nta['member_departures'] / member_pct_by_nta['total_departures'] * 100

print('Neighborhoods with LOWEST member % (likely tourist areas):')
for _, row in member_pct_by_nta.nsmallest(5, 'member_pct').iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: {row['member_pct']:.1f}% members")

print('\nNeighborhoods with HIGHEST member % (likely residential):')
for _, row in member_pct_by_nta.nlargest(5, 'member_pct').iterrows():
    nta_name = stations_with_nta[stations_with_nta['ntacode'] == row['nta']]['ntaname'].iloc[0] if len(stations_with_nta[stations_with_nta['ntacode'] == row['nta']]) > 0 else row['nta']
    print(f"  {nta_name}: {row['member_pct']:.1f}% members")

## 8. Generate Output JSON Files

In [ ]:
# File 1: neighborhoods.json (GeoJSON with metadata)

# Get NTAs that have Citi Bike stations
active_ntas = hourly_stats['nta'].unique()

# Filter and simplify NTA geometries
nta_filtered = nta[nta['ntacode'].isin(active_ntas)].copy()

# Add computed properties
nta_stats = hourly_stats.groupby('nta').agg(
    total_departures=('departures', 'sum'),
    avg_member_pct=('member_pct', 'mean')
).reset_index()

nta_filtered = nta_filtered.merge(
    nta_stats, 
    left_on='ntacode', 
    right_on='nta', 
    how='left'
)

# Add peak hours
nta_filtered = nta_filtered.merge(
    peak_hours,
    left_on='ntacode',
    right_on='nta',
    how='left',
    suffixes=('', '_peak')
)

# Simplify geometries (reduce file size)
nta_filtered['geometry'] = nta_filtered['geometry'].simplify(0.001)

# Select columns for export
export_columns = ['ntacode', 'ntaname', 'boroname', 'total_departures', 'avg_member_pct', 'peak_hour', 'geometry']
nta_export = nta_filtered[[c for c in export_columns if c in nta_filtered.columns]]

# Save as GeoJSON
nta_export.to_file('output/neighborhoods.json', driver='GeoJSON')
print(f'Saved neighborhoods.json ({os.path.getsize("output/neighborhoods.json") / 1024:.1f} KB)')

In [ ]:
# File 2: weekly-patterns.json

# Restructure data: {nta: {day: [{h, dep, arr}]}}
weekly_patterns = {}

for nta in hourly_stats['nta'].unique():
    nta_data = hourly_stats[hourly_stats['nta'] == nta]
    weekly_patterns[nta] = {}
    
    for day in range(7):
        day_data = nta_data[nta_data['day_of_week'] == day].sort_values('hour')
        weekly_patterns[nta][str(day)] = [
            {
                'h': int(row['hour']),
                'dep': int(row['departures']),
                'arr': int(row['arrivals'])
            }
            for _, row in day_data.iterrows()
        ]

with open('output/weekly-patterns.json', 'w') as f:
    json.dump(weekly_patterns, f, separators=(',', ':'))

print(f'Saved weekly-patterns.json ({os.path.getsize("output/weekly-patterns.json") / 1024:.1f} KB)')

In [ ]:
# File 3: story-moments.json

# Get NTA names lookup
nta_names = stations_with_nta.drop_duplicates('ntacode').set_index('ntacode')['ntaname'].to_dict()

# Calculate key statistics for story moments
total_trips = len(trips)

# Morning rush stats (8-9am weekdays)
morning_rush_trips = trips[
    (trips['day_of_week'] < 5) & 
    (trips['hour'] >= 8) & 
    (trips['hour'] <= 9)
]
morning_rush_count = len(morning_rush_trips)

# Peak hour (highest departures overall)
peak_hour_overall = hourly_stats.groupby('hour')['departures'].sum().idxmax()

# Friday night top arrivals
friday_top = night_comparison.nlargest(5, 'friday').index.tolist()
friday_top_names = [nta_names.get(nta, nta) for nta in friday_top]

# Weekend growers
weekend_growers = weekend_comparison.nlargest(5, 'ratio').index.tolist()
weekend_grower_names = [nta_names.get(nta, nta) for nta in weekend_growers]

story_moments = {
    'total_trips': total_trips,
    'year': YEAR,
    'morning_rush': {
        'peak_time': f'{peak_hour_overall}:00',
        'trips_8_9am': morning_rush_count,
        'top_destinations': importers['nta'].head(3).apply(lambda x: nta_names.get(x, x)).tolist()
    },
    'friday_night': {
        'top_arrivals': friday_top_names[:5],
        'peak_hour': 23
    },
    'weekend': {
        'top_growers': weekend_grower_names[:5]
    },
    'neighborhoods': {
        'total_active': len(active_ntas),
        'office_districts': importers['nta'].head(5).apply(lambda x: nta_names.get(x, x)).tolist(),
        'residential': exporters['nta'].head(5).apply(lambda x: nta_names.get(x, x)).tolist()
    }
}

with open('output/story-moments.json', 'w') as f:
    json.dump(story_moments, f, indent=2)

print(f'Saved story-moments.json ({os.path.getsize("output/story-moments.json") / 1024:.1f} KB)')

In [ ]:
# File 4: metadata.json

metadata = {
    'generated_at': pd.Timestamp.now().isoformat(),
    'data_year': YEAR,
    'data_months': list(MONTHS),
    'total_trips': int(total_trips),
    'unique_stations': len(stations),
    'active_neighborhoods': len(active_ntas),
    'files': {
        'neighborhoods.json': f'{os.path.getsize("output/neighborhoods.json") / 1024:.1f} KB',
        'weekly-patterns.json': f'{os.path.getsize("output/weekly-patterns.json") / 1024:.1f} KB',
        'story-moments.json': f'{os.path.getsize("output/story-moments.json") / 1024:.1f} KB'
    }
}

with open('output/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f'Saved metadata.json')
print(f'\n=== SUMMARY ===')
print(f"Total output size: {sum(os.path.getsize(f'output/{f}') for f in os.listdir('output')) / 1024:.1f} KB")

## 9. Download Output Files

In [ ]:
# Create a zip file with all outputs
import shutil

shutil.make_archive('nyc-bike-rhythms-data', 'zip', 'output')

print('Downloading nyc-bike-rhythms-data.zip...')
files.download('nyc-bike-rhythms-data.zip')

---

## Next Steps

1. Download the `nyc-bike-rhythms-data.zip` file
2. Extract to your project's `/data/citibike/` folder
3. Continue with the frontend implementation

The output files should be under 3MB total and ready for static hosting.